In [430]:
# Who is the individual associated with the cryptocurrency industry facing a criminal trial on fraud and conspiracy charges, as reported by both The Verge and TechCrunch, and is accused by prosecutors of committing fraud for personal gain?

# After the TechCrunch report on October 7, 2023, concerning Dave Clark's comments on Flexport, and the subsequent TechCrunch article on October 30, 2023, regarding Ryan Petersen's actions at Flexport, was there a change in the nature of the events reported?
# Do the TechCrunch article on software companies and the Hacker News article on The Epoch Times both report an increase in revenue related to payment and subscription models, respectively?

# Considering the information from a BBC article detailing Sridevi's achievements in the Indian film industry and a Times of India report on her posthumous honors, which single character from a film portrayed by Sridevi has been recognized for its cultural impact and has also been commemorated with a special award after her passing?

In [431]:
# !pip install -r requirements.txt

In [432]:


# import json
# from langchain_core.documents import Document

# # Load JSON data
# def load_data(filepath: str):
#     with open(filepath, 'r') as f:
#         data = json.load(f)
#     return data

# # Prepare documents from JSON for embedding and indexing
# def preprocess_documents(data):
#     documents = []
#     for entry in data:
#         # Create a Document object with page content and metadata
#         documents.append(
#             Document(
#                 page_content=entry['body'],
#                 metadata={
#                     'title': entry['title'] if entry.get('title') else "none",
#                     'author': entry['author'] if entry.get('author') else "none",
#                     'source': entry['source'] if entry.get('source') else "none",
#                     'published_at': entry['published_at'] if entry.get('published_at') else "none",
#                     'category': entry['category'] if entry.get('category') else "none",
#                     'url': entry['url'] if entry.get('url') else "none",
#                 }
#             )
#         )
#     return documents

# # Load and process the corpus.json file
# data = load_data('/content/corpus.json')
# docs = preprocess_documents(data)


In [433]:
# from langchain_chroma import Chroma
# from langchain_core.documents import Document
# from langchain.embeddings import GPT4AllEmbeddings  # Import GPT4All embeddings
# # gpt4all_embd = GPT4AllEmbeddings()
# vectorstore = Chroma.from_documents(docs, GPT4AllEmbeddings())

In [434]:

import zipfile

# Unzip the uploaded file
with zipfile.ZipFile("/content/vector_store.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/chroma_vectorstore")

In [435]:

from langchain.vectorstores import Chroma
from langchain.embeddings import GPT4AllEmbeddings
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from transformers import AutoTokenizer, AutoModelForCausalLM



# Path to the existing Chroma database and vector files
persist_directory = '/content/chroma_vectorstore'  # e.g., 'vector' folder

# Load the existing Chroma vector store (from chroma.sqlite3 and the directory)
vectorstore = Chroma(persist_directory=persist_directory, embedding_function=GPT4AllEmbeddings())




In [ ]:
import json
from transformers import pipeline

# Initialize the zero-shot classification pipeline with the specified model
pipe = pipeline("zero-shot-classification", model="MoritzLaurer/deberta-v3-large-zeroshot-v2.0")

# Define the available categories
available_categories = ["business", "entertainment", "health", "science", "sports", "technology"]

# Function to classify the query using the zero-shot classification pipeline
def classify_query(query: str):
    # Use the pipeline to classify the query
    result = pipe(query, available_categories)

    # Extract the top 2 labels and their scores
    top_categories = result['labels'][:2]  # Get the top 2 categories
    return top_categories

# Prepare a function that takes in a query and returns a structured query with categories and filter
def process_query(query: str) -> dict:
    # Classify the query using zero-shot classification
    categories = classify_query(query)

    # Construct the filter based on the top 2 categories returned
    if categories:
        # Create a filter using only eq comparators for the top 2 categories
        filter_expression = ' or '.join([f'eq("category", "{category}")' for category in categories])
        filter_expression = f'({filter_expression})'

        # Create the output
        output = {
            "query": query,
            "categories": categories,
            "filter": filter_expression
        }
    else:
        # If there's an issue, fall back to return only the query
        output = {"query": query, "filter": ""}

    return output
# query not in train :: Considering the information from an article by The Verge on the latest iCloud security features and a report by Bloomberg on the recent iCloud service outages, which character from the CEO's name of the company responsible for iCloud would be common to both the security feature's codename and the name of the city where the most significant server disruption occurred?
# Example usage
query="Which individual is implicated in both inflating the value of a Manhattan apartment to a figure not yet achieved in New York City's real estate history, according to 'Fortune', and is also accused of adjusting this apartment's valuation to compensate for a loss in another asset's worth, as reported by 'The Age'?"
result = process_query(query)
# print(result)

# Display the structured query with inferred categories and filter
r = result['query'] + result['filter']
# print(r)


d = vectorstore.similarity_search(r, k=3)
print(len(d))
docu = ""
for i in d:
    docu += i.page_content
# print(docu)
label=""
if(len(d)==0):
    label="null_query"
    # output = {
    # "query": query,
    # "answer": "insufficient_information",
    # "question_type": "null_query",
    # "evidence_list":[]
    # }
print(label)

    # Convert to JSON string
    # json_output = json.dumps(output, indent=4)
    # result = generate_json_output()

In [446]:
from transformers import pipeline

# Load the zero-shot classification pipeline with the specified model
classifier = pipeline("zero-shot-classification", model="MoritzLaurer/deberta-v3-large-zeroshot-v2.0")
# Define the possible labels for the first classification
candidate_labels = ["temporal_query", "inference_query", "comparison_query"]

# Classify the query using the first model (DeBERTa)
result = classifier(query, candidate_labels)

# Output the label with the highest score
first_label = result['labels'][0]  # Access the first label

if first_label == "inference_query":
    label = "inference_query"
else:
    # If it's not "inference_query", classify using the second model (RoBERTa)
    classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

    # Define the possible labels for the second classification
    candidate_labels = ["temporal_query", "comparison_query"]

    # Classify the query again
    result = classifier(query, candidate_labels)

    # Output the label with the highest score
    label = result['labels'][0]

# Print the final label
print(label)


temporal_query


In [447]:

from transformers import DistilBertTokenizer
if (label!="null_query"):
    ordered_sentences = docu.split('\n')[:-1]
    model_name = "BlueOrangeDigital/distilbert-cross-segment-document-chunking"
    tokenizer = DistilBertTokenizer.from_pretrained(model_name)

    def right_truncate_sentence(sentence, tokenizer, max_len):
      tokenized = tokenizer.encode(sentence)[1:-1]
      if len(tokenized) > max_len:
          print("cut")
      return tokenizer.decode(tokenized[:max_len])


    def left_truncate_sentence(sentence, tokenizer, max_len):
      tokenized = tokenizer.encode(sentence)[1:-1]
      if len(tokenized) > max_len:
          print("cut")
      return tokenizer.decode(tokenized[-max_len:])

    def bucket_pair(left_sentence, right_sentence, tokenizer, max_len):
      return left_truncate_sentence(left_sentence, tokenizer, max_len) + " [SEP] " + \
          right_truncate_sentence(right_sentence, tokenizer, max_len)

    MAX_LEN = 255
    pairs = [
      bucket_pair(ordered_sentences[i], ordered_sentences[i+1], tokenizer, MAX_LEN)
      for i in range(0, len(ordered_sentences) - 1)
    ]

    from transformers import (
      AutoModelForSequenceClassification,
      TextClassificationPipeline
    )

    model_name = "BlueOrangeDigital/distilbert-cross-segment-document-chunking"

    id2label = {0: "SAME", 1: "DIFFERENT"}
    label2id = {"SAME": 0, "DIFFERENT": 1}

    model = AutoModelForSequenceClassification.from_pretrained(
      model_name,
      num_labels=2,
      id2label=id2label,
      label2id=label2id
    )

    pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer)

    predictions = pipe(pairs)


    n = len(ordered_sentences)
    chunks_breaks = [
      i+1
      for i, pred in enumerate(predictions)
      if pred["label"] != "SAME"
    ]

    chunks = [
      "\n".join(ordered_sentences[i:j])
      for i, j in zip([0] + chunks_breaks, chunks_breaks + [n])
    ]
    print(f"Document split into {len(chunks)} chunks.")



cut
cut
cut
cut
cut
cut
cut
cut
cut
cut
Document split into 25 chunks.


In [448]:
docu=[]
for chunk in chunks:
  if chunk is None:
    continue
  elif chunk == "":
    continue
  docu.append(chunk)


In [449]:
import faiss
import numpy as np
from langchain.embeddings import GPT4AllEmbeddings  # Ensure the import is correct

# Initialize the GPT4All embeddings model
embedder = GPT4AllEmbeddings()

# Initialize FAISS index with the correct embedding dimension
embedding_dim = 384  # Set this to 384, based on the output size of GPT4All
index = faiss.IndexFlatL2(embedding_dim)

# Function to generate embeddings for each chunk and store in FAISS
def store_chunks_in_faiss(chunks):
    for chunk in chunks:
        # Generate embeddings
        embeddings = embedder.embed_documents([chunk])  # Ensure input is a list

        # Assuming embeddings is a list of arrays, extract the first embedding
        embedding = embeddings[0]  # Get the first embedding

        # Convert the embedding to a NumPy array if it's not already
        embedding_np = np.array(embedding, dtype="float32")

        # Ensure the embedding has the correct shape
        if embedding_np.ndim == 1:  # If shape is (384,)
            embedding_np = embedding_np.reshape(1, -1)  # Reshape to (1, 384)

        # Ensure the embedding matches the expected dimension
        if embedding_np.shape[1] != embedding_dim:
            raise ValueError(f"Embedding has incorrect dimensions: {embedding_np.shape[1]} (expected {embedding_dim})")

        # Add embedding to FAISS index
        index.add(embedding_np)

    print(f"Stored {index.ntotal} chunks in FAISS index.")

# Example usage
# Replace 'docu' with your actual list of document chunks
  # Example chunks
store_chunks_in_faiss(docu)

# Optional: Save FAISS index
faiss.write_index(index, "faiss_index.index")


Stored 23 chunks in FAISS index.


In [450]:
import faiss
import numpy as np
from langchain.embeddings import GPT4AllEmbeddings  # Ensure the import is correct

# Initialize the GPT4All embeddings model (if not already initialized)
embedder = GPT4AllEmbeddings()

# Function to perform similarity search
def similarity_search(query, k=5):
    # Generate the embedding for the query
    query_embedding = embedder.embed_documents([query])[0]  # Get the first embedding
    query_embedding_np = np.array(query_embedding, dtype="float32").reshape(1, -1)  # Reshape to (1, 384)

    # Perform the similarity search
    distances, indices = index.search(query_embedding_np, k)  # Get the top k most similar embeddings

    # Return the results
    return distances, indices

# Example usage
#query = "Considering the financial performance outlined in the Bloomberg article and the strategic partnerships mentioned in the Reuters report on Poodle Holdings, which division, represented by its initial, is poised for the most significant expansion in the upcoming fiscal year?"
distances, indices = similarity_search(query, k=5)

# Output the results
# print("Distances:", distances)
# print("Indices:", indices)

# If you want to access the original documents associated with the indices
original_documents = [docu[i] for i in indices[0]]  # Assuming `docu` holds your original chunks
# print("Similar Documents:")
l=""
for doc in original_documents:
    l+=doc+"\n"

if(len(l)>19000):
  distances, indices = similarity_search(query, k=3)

  # Output the results
  # print("Distances:", distances)
  # print("Indices:", indices)

  # If you want to access the original documents associated with the indices
  original_documents = [docu[i] for i in indices[0]]

  l=""
  for doc in original_documents:
    l+=doc+"\n"
# print(len(l))


In [451]:
import json
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer,AutoModelForSeq2SeqLM

# Set up the Hugging Face model


# Convert the documents to a single string
joined_documents = "".join(l)

# Build the prompt based on the classified query type
if label == "inference_query":


  pipe = pipeline("text2text-generation", model="google/t5-v1_1-base")
  tokenizer = AutoTokenizer.from_pretrained("google/t5-v1_1-base")
  model = AutoModelForSeq2SeqLM.from_pretrained("google/t5-v1_1-base")
  prompt_content = (
      "Based on the following documents, please provide a one to two word answer. "
      "Give a straight answer. If there is not enough information, return 'Insufficient Information'.\n\n"
      f"Documents:\n{joined_documents}\n\n"
      f"Query: {query}\nAnswer:"
  )
  inputs = tokenizer(prompt_content, return_tensors="pt")

  # Generate the output
  outputs = model.generate(inputs["input_ids"], max_length=10, num_beams=5, early_stopping=True)

  # Decode the output to text
  answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

  # Print or return the answer
  # print(answer)
elif label=="temporal_query":
    prompt_content = (
        f"Based on the following documents, answer 'No' or 'Yes' to the query. Make sure to be accurate and return 'No' when the evidence is insufficient or the staement in the query is incorrect. Don't include any facts, just the final answer.\n\n"
        f"Documents:\n{joined_documents}\n\n"
        f"Query: {query}\nAnswer:"
    )

    model_name = "meta-llama/Llama-3.2-3B-Instruct"

    candidate_labels = ["yes","no"]

    result = classifier(prompt_content, candidate_labels)

    # Output the label with the highest score
    first_label = result['labels'][0]  # Access the first label
    answer=first_label
else :
    prompt_content = (
      f"Based on the following documents, answer 'YES' or 'NO' to the query. Make sure to be accurate .Return 'No' when the given comparsion is false. If the given comparison is true return 'Yes'. Don't include any facts, just the final answer.\n\n"
      f"Documents:\n{joined_documents}\n\n"
      f"Query: {query}\nAnswer:"
    )

    model_name = "meta-llama/Llama-3.2-3B-Instruct"

    candidate_labels = ["yes","no"]

    result = classifier(prompt_content, candidate_labels)

    # Output the label with the highest score
    first_label = result['labels'][0]  # Access the first label
    answer=first_label
# print(response)

def extract_first_answer(text):
    # Find the index of "Answer:" and get the substring after it until the newline
    answer_index = text.find("Answer:")
    if answer_index != -1:
        # Find the index of the newline character after "Answer:"
        newline_index = text.find("\n", answer_index)
        # If there is no newline, get the rest of the string
        if newline_index == -1:
            return text[answer_index + len("Answer:"):].strip()
        else:
            return text[answer_index + len("Answer:"):newline_index].strip()
    return None

# answer_index = response.find("Answer:")
# answer = extract_first_answer(response)
print(answer)  # Output: This is the answer you are looking for.



yes


In [452]:
def extract_facts(text):
    # Initialize an empty list to store facts
    facts_list = []

    # Split the text into lines
    lines = text.strip().split('\n')

    # Flag to indicate if we are in the facts section
    facts_section = False

    for line in lines:
        # Check if we are starting the facts section
        if line.startswith("Facts:"):
            facts_section = True
            continue  # Skip the "Facts:" line

        # If we are in the facts section, add non-empty lines to the list
        if facts_section and line.strip():
            # Strip leading/trailing whitespace and add to the facts list
            facts_list.append(line.strip().lstrip('-').strip())

    return facts_list




facts = extract_facts(joined_documents)



lists=[]
for i in d:
  x=i.metadata
  for fact in facts:
    if fact in i.page_content:
      x['facts']=fact
  lists.append(x)




In [453]:
import json

# Function to generate the desired JSON format
def generate_json_output():
    # Define the question type (assuming it's already set to some label)
    question_type = label

    # Check if the answer is "Insufficient Information"
    if answer == "Insufficient Information.":
        evidence_list = []  # Empty evidence list
        question_type = "null_query"  # Change the question type to "null_query"
    else:
        evidence_list = lists  # Use the lists variable as evidence

    # Create the final output dictionary
    output = {
        "query": query,
        "answer": answer,
        "question_type": question_type,
        "evidence_list": evidence_list
    }

    # Convert to JSON string
    json_output = json.dumps(output, indent=4)
    return json_output

# Generate and print the JSON output
result = generate_json_output()
print(result)


{
    "query": "After the TechCrunch report on October 7, 2023, concerning Dave Clark's comments on Flexport, and the subsequent TechCrunch article on October 30, 2023, regarding Ryan Petersen's actions at Flexport, was there a change in the nature of the events reported?",
    "answer": "yes",
    "question_type": "temporal_query",
    "evidence_list": [
        {
            "author": "none",
            "category": "business",
            "published_at": "2023-10-02T17:46:00+00:00",
            "source": "Cnbc | World Business News Leader",
            "title": "The inside story of Dave Clark's tumultuous last days at Flexport",
            "url": "https://www.cnbc.com/2023/10/02/the-inside-story-of-dave-clarks-tumultuous-last-days-at-flexport.html"
        },
        {
            "author": "Rebecca Bellan",
            "category": "technology",
            "published_at": "2023-10-22T19:15:29+00:00",
            "source": "TechCrunch",
            "title": "Tesla \u2018digs its ow